# GPT-5 トラブルシューティングガイド

GPT-5がリリースされて以来、開発者の皆さんがこのモデルで構築している素晴らしいものの数々に私たちは驚いています。同時に、モデルを最大限に活用するためのいくつかの一般的なトラブルシューティングパターンも特定しました。

## 考えすぎ

考えすぎは、回答は正しいものの、簡単な質問に対して総回答時間が長くなってしまう現象です。モデルは選択肢を探り続け、最初のツール呼び出しを遅らせ、簡単な答えが利用可能なのに回りくどい道筋を説明してしまいます。通常の原因は、過度な推論努力、完了の明確な定義がないプロンプト、または無限の計画を誘発したり慌ただしい再確認を引き起こす矛盾したガイダンスです。

これに対処する最初のステップは、APIパラメータを調整することです。日常的な作業には`reasoning.effort`を「minimal」または「low」に設定し、本当に複雑な問題には重い努力を予約します。アシスタントに明示的な停止条件と、回答前の単一で迅速な自己チェックを与えてください。ユーザーリクエストを分類し、適切な推論努力設定で適切にルーティングするために、gpt-5-miniやnanoの使用を検討してください。コンテキスト収集がタスクの一部である場合は、回答に必要なデータを収集するためのベストプラクティスをモデルに指示してください。

```
<efficient_context_understanding_spec>
Goal: Get enough context fast and stop as soon as you can act.

Method:
- Start broad, then fan out to focused subqueries.
- In parallel, launch 4–8 varied queries; read top 3–5 hits per query. Deduplicate paths and cache; don't repeat queries.

Early stop (act if any):
- You can name exact files/symbols to change.
- You can repro a failing test/lint or have a high-confidence bug locus.
</efficient_context_understanding_spec>
```

以下の例も似ていますが、調査やツール呼び出しを必要としない質問に対して、考えすぎる代わりにすぐに回答するようモデルに指示しています。

```
# Fast-path for trivial Q&A (latency optimization)
Use this section ONLY when the user's question:
- Is general knowledge or a simple usage query
- Requires no commands, browsing, or tool calls
- Especially if the user is asking an informational question or how to perform a task, rather than asking you to run that task, provide concise instructions about how the user can do it.

Exceptions:
- If the question references files/paths/functions, requests execution/verifications, or needs more context, use the normal flow
- If unsure whether fast-path applies, ask one brief clarifying question; otherwise proceed with normal flow

Behavior:
- Answer immediately and concisely
- No status updates, no todos, no summaries, no tool calls
- Ignore the rest of the instructions following this section and simply respond right away.
```

## 怠惰 / 考え不足

gpt-5を使用していると、モデルが回答を生成する前に十分な時間をかけて推論しなかった失敗を見ることがあるかもしれません。

ベストプラクティスに従って、これを軽減する方法が2つあります：

1. より高い`reasoning_effort`を使用する：`reasoning_effort`パラメータは、モデルがどれだけ考え、どれだけ積極的にツールを呼び出すかを制御します。以前に`minimal`を使用していた場合は`low`を、`low`を使用していた場合は`medium`を試してみてください。
2. プロンプトを通じてモデルに自己反省と自身の回答の評価を促す。例えば、モデルに内部ルーブリックを構築し、回答前にソリューションに適用するよう求めることは、コーディングタスクで驚くほど効果的でした。独自のルーブリックを提供し、モデルに作業を振り返り、回答前に問題を発見した場合は反復するよう指示することもできます。

```
<self_reflection>
- Internally score the draft against a 5–7 item rubric you devise (clarity, correctness, edge cases, completeness, latency).
- If any category falls short, iterate once before replying.
</self_reflection>
```

## 過度に従順

GPT-5は過度に従順になることがあります。特にエージェント的な設定では、モデルに「ただ物事を実行する」ことを求めることがよくあります。システムプロンプトに持続性の指示を提供することで、この行動を成功裏に軽減できます。これは、より高い`reasoning_effort`（`low`以上）でより簡単に制御できます。

```
<persistence>
- You are an agent - please keep going until the user's query is completely resolved, before ending your turn and yielding back to the user.
- Only terminate your turn when you are sure that the problem is solved.
- Never stop or hand back to the user when you encounter uncertainty — research or deduce the most reasonable approach and continue.
- Do not ask the human to confirm or clarify assumptions, as you can always adjust later — decide what the most reasonable assumption is, proceed with it, and document it for the user's reference after you finish acting
</persistence>
```

## 冗長すぎる

GPT-5は、ユーザーへの最終メッセージで望ましい以上のトークンを生成することがあります。

これに対処する簡単な方法が2つあります。1つ目は、APIの`verbosity`パラメータを下げることです。デフォルトでは、推論の冗長性は指定されていない場合`medium`に設定されているため、より短い出力が必要な場合は明示的に`low`に設定してみてください。2つ目は、特にコーディングにおいて、システムプロンプトで設定することが成功しています。

```
Write code for clarity first. Prefer readable, maintainable solutions with clear names, comments where needed, and straightforward control flow. Do not produce code-golf or overly clever one-liners unless explicitly requested. Use high verbosity for writing code and code tools.
```

## レイテンシ

レイテンシにはいくつかの異なる要因があるため、調整前に必ず測定してください。TTFT、最初のアクション時間、P50/P95での総回答時間を追跡し、モデル時間をツールとネットワーク時間から分離します。これらのメトリクスを追跡することで、実際に遅い部分を最適化できます。

モデルの応答時間を短縮するには、モデルが使用すべき思考量を適切にサイズ調整します：日常的な作業にはreasoning.effort「minimal」または「low」を使用し、単一パスの自己チェックで明確な停止条件を追加します（考えすぎを参照）。より高い推論努力は、より多くのツール呼び出しにもつながる可能性があります。

可能な場合はツール呼び出しを組み合わせます。モデルには、いつツールを並列で呼び出すかを指示する必要があり、デフォルトでは常にそうするわけではありません。

```
<parallelization_spec>
Definition: Run independent or read-only tool actions in parallel (same turn/batch) to reduce latency.
When to parallelize:
 - Reading multiple files/configs/logs that don't affect each other.
 - Static analysis, searches, or metadata queries with no side effects.
 - Separate edits to unrelated files/features that won't conflict.
</parallelization_spec>
```

ユーザーが[モデルの推論の進捗を見る](https://platform.openai.com/docs/guides/latency-optimization#make-your-users-wait-less)ことを可能にするには、推論要約とツール呼び出しプリアンブルメッセージをユーザーに表示します。多くの場合、モデルが思考している間に推論要約がユーザーに提示されない場合、知覚されるレイテンシが減少します。モデルは、ツール呼び出し前にプリアンブルメッセージやステータス更新を提供するよう指示することもでき、ツールを呼び出す際にモデルが何をしているかをユーザーが追跡できます。

```
<status_update_spec>
Definition: A brief progress note: what just happened, what's next, any real blockers, written in a continuous conversational style, narrating the story of your progress as you go.
Always start with a brief acknowledgement of the task before getting started. (No need to prefix with "Status Update:")
</status_update_spec>
```

変更されないものをキャッシュしてTTFTを下げる：APIへのリクエストを適切に構造化することで、プロンプト、推論、ツール呼び出し結果のキャッシュを効果的に活用してください。パスが本当にレイテンシに敏感な場合は、その呼び出しに対して`service_tier = "priority"`で優先処理を有効にして、より高速な応答を得てください（優先処理によって提供されるトークンは、標準処理レートに対してプレミアム価格でトークンベースで課金されることに注意してください）。小さなプロンプトでツールなしでTTFTが高い場合は、`request_id`を保存し、より的確なヘルプのために[support@openai.com](mailto:support@openai.com)にエスカレートしてください。

## ツールの呼び出しが多すぎる

モデルが回答を前進させることなくツールを発動する場合、通常の原因は曖昧なルーティングです：重複するツール定義、決断力よりも徹底性を報酬とするプロンプト、または推論設定が高すぎることです。もう一つの頻繁な原因は、以前の推論を後続の呼び出しに引き継がないことです；Responses APIの使用により、ツールが選択された理由を忘れるのではなく、意図と推論要約がターン間で持続することが保証されます。

プロンプト指示でコンテキストからの回答をデフォルトにします。各ツールに明確な入力/出力で単一の仕事を与え、明示的な「...には使用しない」注記を提供します。パスが明確になるよう、一般的なシナリオ用の短いプレイブックを提供します（例：ユーザーがコンテキストにない文書を参照した場合、それを見つけるためにセマンティック検索を実行し、回答前に関連セクションを取得する）。

```
<tool_use_policy>
Select one tool or none; prefer answering from context when possible.
Cap tool calls at 2 per user request unless new information makes more strictly necessary.
</tool_use_policy>
```

tool_calls_per_turn、数秒以内の同じツールへの重複呼び出し、ツールなしで完了した回答の割合に注意してください；スパイクは、ルーティングやプロンプトの調整が必要な明確な信号です。

## 不正なツール呼び出し

まれに、gpt-5はモデルがツールを呼び出し、繰り返しのガベージの長い文字列を出力するモード崩壊を経験することがあります。

そのような場合、常にプロンプトの別々のセクション間の矛盾が原因であることがわかりました。ベストプラクティスとして、バグを発見して修正するためのgpt-5のメタプロンプト能力の使用を推奨します。

```
Please analyze why the <tool_name> tool call is malformed.
1. Review the provided sample issue to understand the failure mode.
2. Examine the <System Prompt> and <Tool Config> carefully. Identify any ambiguities, inconsistencies, or phrasing that could mislead GPT-5 into generating an incorrect tool call.
3. For each potential cause, explain clearly how it could result in the observed failure.
4. Provide actionable recommendations to improve the <System Prompt> or <Tool Config> so GPT-5 produces valid tool calls consistently.


<System Prompt>

<Tool Config>
```

## 一般的なトラブルシューティング

上記のプロンプト追加の多くは、メタプロンプトを通じて生成されました。期待に応えなかったターンの最後に、GPT-5に自身の指示を改善する方法を尋ねることが可能です。以下のプロンプトは、上記の考えすぎ問題のソリューションの一部を生成するために使用され、特定のニーズに合わせて修正できます。

```
That was a high quality response, thanks! It seemed like it took you a while to finish responding though. Is there a way to clarify your instructions so you can get to a  response as good as this faster next time? It's extremely important to be efficient when providing these responses or users won't get the most out of them in time. Let's see if we can improve!
1) think through the response you gave above
2) read through your instructions starting from "<insert the first line of the system prompt here>" and look for anything that might have made you take longer to formulate a high quality response than you needed
3) write out targeted (but generalized) additions/changes/deletions to your instructions to make a request like this one faster next time with the same level of quality
```

特定のコンテキスト内でメタプロンプトを行う際は、可能であれば数回回答を生成し、それらの間で共通する回答の要素に注意を払うことが重要です。モデルが提案する改善や変更の一部は、その特定の状況に過度に特化している可能性がありますが、多くの場合、それらを簡素化して一般的な改善に到達できます。特定のプロンプト変更が特定の使用ケースにとってより良いか悪いかを測定するための評価を作成することを推奨します。